# SQL_ONE - Análise de Dados com SQL
## Aula 3 - Coalesce, Cast, First Value, Last Value, With, Subquery

### 1 - Base de dados pilotos_nan
#### Acessando a base de dados no github com PANDAS, transformando em dataframe SPARK, escrevendo uma TABLE para SQL.

In [0]:
%py
# 1 - Importando o pandas
import pandas as pd

# Acessando a base de dados df_pilotos
df_pilotos = pd.read_csv("https://raw.githubusercontent.com/wallacecarlis/arquivos/main/base_pilotos.csv")

# Criando um spark dataframe
df_pyspark_pilotos = spark.createDataFrame(df_pilotos)

# Criando uma tabela delta
df_pyspark_pilotos.write.format("delta").mode("overwrite").saveAsTable("pilotos_nan")

### 2 - Select, Order By
#### Vamos visualizar a base com SELECT e ORDER BY para identificar os valores nulos (NULL).

In [0]:
-- 2 - Visualizando os dados da tabela pilotos_nan com Order by
SELECT * FROM pilotos_nan
ORDER BY id;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
2,Rubens Barrichello,50.0,São Paulo,Jordan
3,Lewis Hamilton,37.0,Stevenage,McLaren
4,Michael Schumacher,53.0,Hürth,null
5,Max Verstappen,25.0,Hasselt,null
6,Nelson Piquet,null,null,Ensign
8,Emerson Fittipaldi,null,null,Lotus
9,José Carlos Pace,null,null,Willys


### 3 - Cast, Avg
#### Uma vez que identificamos a existência de valores nulos, podemos utilizar algumas técnicas para realizar seus preenchimentos. Para os valores numéricos da coluna IDADE, utilizaremos CAST e AVG para preencher com a média.

In [0]:
-- 3 - Verificando a média da coluna idade com as funções cast e avg
SELECT 
  AVG(idade) AS idade_media,
  CAST(AVG(idade) AS INTEGER) AS idade 
FROM pilotos_nan 
WHERE idade IS NOT NULL

idade_media,idade
45.4,45


### 4 - Coalesce, first_value
#### Para as colunas categóricas, utilizaremos COALESCE e FIRST VALUE para preencher com o primeiro valor não nulo.

In [0]:
-- 4 - Preenchendo a idade com o valor médio da coluna, naturalidade e equipe com o primeiro valor não nulo
-- ordenando pelo id
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, FIRST_VALUE(naturalidade) OVER (ORDER BY id)) AS naturalidade,

  COALESCE(equipe, FIRST_VALUE(equipe) OVER (ORDER BY id)) AS equipe

FROM pilotos_nan;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
2,Rubens Barrichello,50.0,São Paulo,Jordan
3,Lewis Hamilton,37.0,Stevenage,McLaren
4,Michael Schumacher,53.0,Hürth,Toleman
5,Max Verstappen,25.0,Hasselt,Toleman
6,Nelson Piquet,45.0,São Paulo,Ensign
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys


### 5 - Coalesce, First Value, Over + Order By
#### Podemos utilizar a cláusula OVER para ordenar o preenchimento a partir de determinada coluna.

In [0]:
-- 5 - Preenchendo a idade com o valor médio da coluna, naturalidade e equipe com o primeiro valor não nulo
-- ordenando pelo nome
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, FIRST_VALUE(naturalidade) OVER (ORDER BY nome)) AS naturalidade,

  COALESCE(equipe, FIRST_VALUE(equipe) OVER (ORDER BY nome)) AS equipe
  
FROM pilotos_nan;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys
3,Lewis Hamilton,37.0,Stevenage,McLaren
5,Max Verstappen,25.0,Hasselt,Toleman
4,Michael Schumacher,53.0,Hürth,Toleman
6,Nelson Piquet,45.0,São Paulo,Ensign
2,Rubens Barrichello,50.0,São Paulo,Jordan


### 6 - Coalesce, First Value, Last Value, Over + Order By
#### Podemos alterar os preenchimentos utilizando FIRST VALUE para uma coluna e LAST VALUE para outra.

In [0]:
-- 6 - Preenchendo com o valor médio para idade, primeiro valor para naturalidade, último valor para equipe
-- ordenando pelo id
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, FIRST_VALUE(naturalidade) OVER (ORDER BY id)) AS naturalidade,

  COALESCE(equipe, LAST_VALUE(equipe) OVER (ORDER BY id)) AS equipe

FROM pilotos_nan;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
2,Rubens Barrichello,50.0,São Paulo,Jordan
3,Lewis Hamilton,37.0,Stevenage,McLaren
4,Michael Schumacher,53.0,Hürth,null
5,Max Verstappen,25.0,Hasselt,null
6,Nelson Piquet,45.0,São Paulo,Ensign
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys


In [0]:
-- 7 - Preenchendo com o valor médio para idade, primeiro valor para naturalidade, último valor para equipe
-- ordenando pelo nome
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, first_value(naturalidade) OVER (ORDER BY nome)) AS naturalidade,

  COALESCE(equipe, last_value(equipe) OVER (ORDER BY nome)) AS equipe

FROM pilotos_nan;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys
3,Lewis Hamilton,37.0,Stevenage,McLaren
5,Max Verstappen,25.0,Hasselt,null
4,Michael Schumacher,53.0,Hürth,null
6,Nelson Piquet,45.0,São Paulo,Ensign
2,Rubens Barrichello,50.0,São Paulo,Jordan


### 8 - Coalesce, Last Value, Over + Order By
#### Podemos preencher somente com LAST VALUE.

In [0]:
-- 8 - Preenchendo a idade com a média, naturalidade e equipe com o último valor
-- ordenando pelo id
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, LAST_VALUE(naturalidade) OVER (ORDER BY id)) AS naturalidade,

  COALESCE(equipe, LAST_VALUE(equipe) OVER (ORDER BY id)) AS equipe
FROM pilotos_nan

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
2,Rubens Barrichello,50.0,São Paulo,Jordan
3,Lewis Hamilton,37.0,Stevenage,McLaren
4,Michael Schumacher,53.0,Hürth,null
5,Max Verstappen,25.0,Hasselt,null
6,Nelson Piquet,45.0,null,Ensign
8,Emerson Fittipaldi,45.0,null,Lotus
9,José Carlos Pace,45.0,null,Willys


### 9 - Coalesce, Last Value, Rows Between
#### Podemos corrigir a deficiência do LAST VALUE aplicando o ROWS BETWEEN.

In [0]:
-- 9 - Preenchendo a idade com a média, naturalidade e equipe com o último valor
-- ordenando pelo nome e rows between
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, LAST_VALUE(naturalidade) 
    OVER (ORDER BY nome ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) AS naturalidade,

  COALESCE(equipe, LAST_VALUE(equipe) 
    OVER (ORDER BY nome ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) AS equipe
    
FROM pilotos_nan

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys
3,Lewis Hamilton,37.0,Stevenage,McLaren
5,Max Verstappen,25.0,Hasselt,Jordan
4,Michael Schumacher,53.0,Hürth,Jordan
6,Nelson Piquet,45.0,São Paulo,Ensign
2,Rubens Barrichello,50.0,São Paulo,Jordan


### 10 - Coalesce, Mode (count + group by)
#### Podemos inclusive preencher com o valor que mais se repete, no caso, a moda (MODE).

In [0]:
-- 10 - Preenchendo a idade com a média, naturalide com a moda, equipe com o último valor
-- ordenando pelo nome com rows between
SELECT 
  id, 
  nome,
  COALESCE(idade, (SELECT CAST(AVG(idade) AS INTEGER) FROM pilotos_nan)) AS idade,

  COALESCE(naturalidade, (SELECT naturalidade FROM 
                            (SELECT 
                              naturalidade, 
                              COUNT(*) AS freq 
                            FROM pilotos_nan 
                            WHERE naturalidade IS NOT NULL 
                            GROUP BY naturalidade 
                            ORDER BY freq DESC 
                            LIMIT 1) moda)) AS naturalidade,

  COALESCE(equipe, LAST_VALUE(equipe) 
    OVER (ORDER BY nome ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING)) AS equipe

FROM pilotos_nan;

id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys
3,Lewis Hamilton,37.0,Stevenage,McLaren
5,Max Verstappen,25.0,Hasselt,Jordan
4,Michael Schumacher,53.0,Hürth,Jordan
6,Nelson Piquet,45.0,São Paulo,Ensign
2,Rubens Barrichello,50.0,São Paulo,Jordan


### 11 - With, subqueries, crossjoin
#### Para um trabalho bem profissional, podemos utilizar as estruturas WITH, SUBQUERY e CROSSJOIN junto com a média, o LAST VALUE com ROWS BETWEEN, e COALESCE para preencher os valores nulos.

In [0]:
-- 11 - Criando consultas com with, subqueries e crossjoin
-- Preenchendo a idade com a média, naturalide com a moda, equipe com o último valor
-- ordenando pelo nome com rows between
WITH

MEDIA_IDADE AS (
    SELECT CAST(AVG(idade) AS INTEGER) AS idade_media
    FROM pilotos_nan
    WHERE idade IS NOT NULL
),

MODA_NATURALIDADE AS (
    SELECT naturalidade
    FROM (
        SELECT 
            naturalidade, 
            COUNT(*) AS freq
        FROM pilotos_nan
        WHERE naturalidade IS NOT NULL
        GROUP BY naturalidade
        ORDER BY freq DESC
        LIMIT 1
    ) moda_do_from
),

LAST_EQUIPE AS (
    SELECT
        LAST_VALUE(equipe)
            OVER (ORDER BY nome ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS ultima_equipe
    FROM pilotos_nan
    LIMIT 1
)

SELECT 
    id,
    nome,
    COALESCE(pn.idade, mi.idade_media) AS idade,
    COALESCE(pn.naturalidade, mn.naturalidade) AS naturalidade,
    COALESCE(pn.equipe, ef.ultima_equipe) AS equipe
FROM pilotos_nan pn
CROSS JOIN MEDIA_IDADE mi
CROSS JOIN MODA_NATURALIDADE mn
CROSS JOIN LAST_EQUIPE ef;


id,nome,idade,naturalidade,equipe
1,Ayrton Senna,62.0,São Paulo,Toleman
8,Emerson Fittipaldi,45.0,São Paulo,Lotus
9,José Carlos Pace,45.0,São Paulo,Willys
3,Lewis Hamilton,37.0,Stevenage,McLaren
5,Max Verstappen,25.0,Hasselt,Jordan
4,Michael Schumacher,53.0,Hürth,Jordan
6,Nelson Piquet,45.0,São Paulo,Ensign
2,Rubens Barrichello,50.0,São Paulo,Jordan
